In [2]:
%cd C:\Users\Tophe\PycharmProjects\underdog-devs-ds-a\

C:\Users\Tophe\PycharmProjects\underdog-devs-ds-a


In [3]:
# !dir

In [4]:
import pandas as pd
import altair as alt
from pandas import DataFrame

In [5]:
from app.data import MongoDB

In [6]:
db = MongoDB()

In [7]:
db

In [8]:
mentee_referrals = pd.DataFrame(
    db.projection('Mentees', {'is_active': True, 'validate_status': 'approved'}, {'referred_by': True}))

In [9]:
mentees = pd.DataFrame(db.read('Mentees'))

In [10]:
mentees['role'] = 'mentee'

In [11]:
mentees_activity = mentees.filter(['role', 'is_active'])

In [12]:
mentees_activity

,role,is_active
0,mentee,True
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,False
...,...,...
96,mentee,True
97,mentee,True
98,mentee,True
99,mentee,False


In [13]:
mentees_activity

,role,is_active
0,mentee,True
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,False
...,...,...
96,mentee,True
97,mentee,True
98,mentee,True
99,mentee,False


In [14]:
mentors = pd.DataFrame(db.read('Mentors'))

In [15]:
mentors = mentors.query("first_name != 'string'")

In [16]:
mentors['role'] = 'mentor'

In [17]:
mentors_activity = mentors.filter(['role', 'is_active'])

In [18]:
mentors_activity

,role,is_active
0,mentor,True
1,mentor,True
2,mentor,True
3,mentor,True
4,mentor,True
5,mentor,True
6,mentor,True
7,mentor,True
8,mentor,True
9,mentor,True


In [19]:
activity = pd.concat([mentees_activity, mentors_activity])

In [20]:
activity

,role,is_active
0,mentee,True
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,False
...,...,...
15,mentor,True
16,mentor,True
17,mentor,True
18,mentor,True


In [21]:
meetings_df = pd.DataFrame(db.read("Meetings"))

In [22]:
meetings_missed = meetings_df.filter(['meeting_missed_by_mentee'])

In [23]:
missed = alt.Chart(meetings_missed).mark_bar().encode(x = 'meeting_missed_by_mentee', y = 'count(meeting_missed_by_mentee)')

In [24]:
missed

alt.Chart(...)

In [25]:
activity_vis = alt.Chart(activity).mark_bar().encode(x = 'is_active', y = 'count(is_active)', color = 'role')

In [26]:
activity_vis

alt.Chart(...)

In [27]:
alt.Chart(meetings_df).mark_bar().encode(x='meeting_topic', y= 'count(meeting_topic)')

alt.Chart(...)

In [28]:
# meetings_df

In [29]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")

In [30]:
def bar_chart(df, column_1, column_2):
    return alt.Chart(
        df,
        title=f"{title_fix(column_1)}",
    ).mark_bar().encode(
        x=alt.X(column_1, title=title_fix(column_1), sort="-y"),
        y=alt.Y(f"count({column_1})"),
        color=alt.Color(column_2, title=title_fix(column_2)),
        tooltip=alt.Tooltip([column_2, column_1, f"count({column_1})"])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )

In [31]:
bar_chart(meetings_df, 'meeting_topic','count(meeting_topic)')

alt.Chart(...)

In [32]:
bar_chart(activity, 'is_active', 'role')

alt.Chart(...)

In [33]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")

def meeting_chart(df, column_1, column_2):
    return alt.Chart(
        df,
        title=f"{title_fix(column_1)}",
    ).mark_bar().encode(
        x=alt.X(column_1, title=title_fix(column_1), sort="-y"),
        y=alt.Y(f"count({column_1})"),
        color=alt.Color(column_1, title=title_fix(f"Legend")),
        tooltip=alt.Tooltip([column_2, column_1, f"count({column_1})"])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )

In [34]:
meeting_chart(meetings_missed, 'meeting_missed_by_mentee', 'count(meeting_missed_by_mentee)')

alt.Chart(...)

In [35]:
from fastapi import APIRouter

In [36]:
Router = APIRouter(
    tags=["Graph Operations"],
)
Router.db = MongoDB()

In [37]:
@Router.get("/graph/meeting-topics")
async def meeting_topics():
    """Meeting subjects, stacked bar chart
    <pre><code>
    @return JSON{Altair.Chart}</pre></code>"""
    description = "This graph shows something."
    return {
        "graph": meeting_chart(
            df_meeting_topics(Router.db),
            "meeting_topic",
            "count(meeting_topic)",
        ).to_dict(),
        "description": description,
    }

In [38]:
import requests, altair
url = "http://127.0.0.1:8000"
graph = requests.get(f"{url}/graph/meeting-topics").json()
altair.Chart.from_dict(graph['graph'])

alt.Chart(...)

In [39]:
url = "http://127.0.0.1:8000"
graph = requests.get(f"{url}/graph/meetings-missed").json()
altair.Chart.from_dict(graph['graph'])

alt.Chart(...)

In [40]:
url = "http://127.0.0.1:8000"
graph = requests.get(f"{url}/graph/activity-status").json()
altair.Chart.from_dict(graph['graph'])

alt.Chart(...)